<a href="https://colab.research.google.com/github/manulon/TP2Simulacion/blob/main/Ejercicio_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EJERCICIO 4

Implementar las dos alternativas del ejercicio 1 utilizando SimPy

In [2]:
!pip install simpy
import simpy
import numpy as np
import random


## Alternativa 1 (Bases Distribuidas)

In [5]:
class llamada:
	def __init__(self,env,tiemposLlamadas):
		self.env = env
		self.demora = []
		self.call_proc = env.process(self.calcular_demora(tiemposLlamadas))

	def dividirMuestrasPorBase(self,tiemposMuestras):
		tiemposMuestrasPorBase = {}
		tiemposMuestrasPorBase[0] = []
		tiemposMuestrasPorBase[1] = []
		for muestra in tiemposMuestras:
			u = np.random.rand()
			if u < 0.6:
				base = 0
			else:
				base = 1
			tiemposMuestrasPorBase[base].append(muestra)
		return tiemposMuestrasPorBase

	def calcular_demora(self,tiemposMuestras):
		muestrasPorBase = self.dividirMuestrasPorBase(tiemposMuestras)
		tiemposMuestrasB1 = muestrasPorBase[0]
		tiemposMuestrasB2 = muestrasPorBase[1]

		tiemposEsperaB1 = np.random.exponential(0.7, len(tiemposMuestrasB1))
		demoraBase1 = self.env.process(self.llamar(tiemposMuestrasB1, tiemposEsperaB1, len(tiemposMuestrasB1)))		

		tiemposEsperaB2 = np.random.exponential(0.95, len(tiemposMuestrasB2))
		demoraBase2 = self.env.process(self.llamar(tiemposMuestrasB2, tiemposEsperaB2, len(tiemposMuestrasB2)))
		
		yield demoraBase2 & demoraBase1 #Bases distribuidas!!! Se ejecutan a la vez.

		print("El tiempo total en horas es: "+ str(env.now/3600))
		print("El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es {:.3f} ".format(self.demora.mean()))
		print("La fraccion de las solicitudes que no esperaron es: {:.3f}".format((100000 - np.count_nonzero(self.demora))/100000))
		print("La tasa de finalizacion de consultas por segundo es: {:.3f}\n".format(100000/(env.now)))

	def llamar(self,tiemposLlamadas,duracionLlamadas,n):
		tiemposEsperaFinal = np.zeros(n)
		tiempoDeEsperaAnterior = 0
		j = 1
		yield self.env.timeout(duracionLlamadas[0])

		while j < n:
			deltaT = tiemposLlamadas[j] - tiemposLlamadas[j - 1] - tiempoDeEsperaAnterior
			tiempoDeEspera = duracionLlamadas[j - 1] - deltaT
			if tiempoDeEspera <= 0:
				tiempoDeEspera = 0

			yield self.env.timeout(duracionLlamadas[j])
			if j != (n-1):
				deltaT2 = tiemposLlamadas[j+1] - (tiemposLlamadas[j] + duracionLlamadas[j] + tiempoDeEspera)
				if deltaT2 > 0:
					yield self.env.timeout(deltaT2)

			tiemposEsperaFinal[j] = tiempoDeEspera
			tiempoDeEsperaAnterior = tiempoDeEspera
			j += 1
		self.demora = np.concatenate((self.demora, tiemposEsperaFinal))


In [6]:

print("ALTERNATIVA 1\n")

print("Mu = 1\n")
z_llamadas = np.random.exponential(1, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas)
env.run()

print("---------------------------------------")
print("Mu = 2\n")
z_llamadas = np.random.exponential(2, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas)
env.run()

print("---------------------------------------")
print("Mu = 4\n")
z_llamadas = np.random.exponential(4, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas)
env.run()

ALTERNATIVA 1

Mu = 1

El tiempo total en horas es: 27.829554226000656
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 0.522 
La fraccion de las solicitudes que no esperaron es: 0.598
La tasa de finalizacion de consultas por segundo es: 0.998

---------------------------------------
Mu = 2

El tiempo total en horas es: 55.81295719039706
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 0.204 
La fraccion de las solicitudes que no esperaron es: 0.798
La tasa de finalizacion de consultas por segundo es: 0.498

---------------------------------------
Mu = 4

El tiempo total en horas es: 111.30887050668834
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 0.089 
La fraccion de las solicitudes que no esperaron es: 0.899
La tasa de finalizacion de consultas por segundo es: 0.250



## Alternativa 2 (Base Central)

In [3]:

class llamada:
  def __init__(self,env,tiemposLlamadas,duracionLlamadas):
    self.env = env
    self.call_proc = env.process(self.llamar(tiemposLlamadas,duracionLlamadas))

  def llamar(self,tiemposLlamadas,duracionLlamadas):
    n = 100000
    tiemposEsperaFinal = np.zeros(n)
    tiempoDeEsperaAnterior = 0
    j = 1
    yield self.env.timeout(duracionLlamadas[0])

    while j < n:
      deltaT = tiemposLlamadas[j] - tiemposLlamadas[j - 1] - tiempoDeEsperaAnterior
      tiempoDeEspera = duracionLlamadas[j - 1] - deltaT
      if tiempoDeEspera <= 0:
        tiempoDeEspera = 0

      yield self.env.timeout(duracionLlamadas[j])
      if j != (n-1):
        deltaT2 = tiemposLlamadas[j+1] - (tiemposLlamadas[j] + duracionLlamadas[j] + tiempoDeEspera)
        if deltaT2 > 0:
          yield self.env.timeout(deltaT2)

      tiemposEsperaFinal[j] = tiempoDeEspera
      tiempoDeEsperaAnterior = tiempoDeEspera
      j += 1

    print("El tiempo total en horas es: "+ str(env.now/3600))
    print("El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es {:.3f} ".format(tiemposEsperaFinal.mean()))
    print("La fraccion de las solicitudes que no esperaron es: {:.3f}".format((n - np.count_nonzero(tiemposEsperaFinal))/n))
    print("La tasa de finalizacion de consultas por segundo es: {:.3f}\n".format(n/(env.now)))

In [4]:
print("ALTERNATIVA 2\n")
tiemposC2 = np.random.exponential(0.8, 100000)

print("Mu = 1\n")
z_llamadas = np.random.exponential(1, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas,tiemposC2)
env.run()

print("---------------------------------------")
print("Mu = 2\n")
z_llamadas = np.random.exponential(2, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas,tiemposC2)
env.run()

print("---------------------------------------")
print("Mu = 4\n")
z_llamadas = np.random.exponential(4, 100000)
tiemposLlamadas = np.concatenate(([0], np.cumsum(z_llamadas)), axis=None)

env = simpy.Environment()
my_llamada = llamada(env,tiemposLlamadas,tiemposC2)
env.run()

ALTERNATIVA 2

Mu = 1

El tiempo total en horas es: 27.77349295147278
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 3.205 
La fraccion de las solicitudes que no esperaron es: 0.197
La tasa de finalizacion de consultas por segundo es: 1.000

---------------------------------------
Mu = 2

El tiempo total en horas es: 55.6056060845947
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 0.533 
La fraccion de las solicitudes que no esperaron es: 0.602
La tasa de finalizacion de consultas por segundo es: 0.500

---------------------------------------
Mu = 4

El tiempo total en horas es: 111.24222825530009
El promedio del tiempo de demora entre que llega una solicitud y puede ser procesada es 0.200 
La fraccion de las solicitudes que no esperaron es: 0.801
La tasa de finalizacion de consultas por segundo es: 0.250

